<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

### Track Experiments

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [9]:
# Initialize MLflow
mlflow.set_tracking_uri("http://localhost:5001")
mlflow.set_experiment("Anomaly Detection_new")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/05/11 10:06:48 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection_new' does not exist. Creating a new experiment.
2025/05/11 10:06:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5001/#/experiments/596555950902332859/runs/3b43beb6cc5c4a6da44122b37909469a
🧪 View experiment at: http://localhost:5001/#/experiments/596555950902332859


2025/05/11 10:06:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5001/#/experiments/596555950902332859/runs/0f199cb6ac6040889869d93056cee692
🧪 View experiment at: http://localhost:5001/#/experiments/596555950902332859


2025/05/11 10:06:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5001/#/experiments/596555950902332859/runs/643e31b5fe224037a1bb903b77dbfd90
🧪 View experiment at: http://localhost:5001/#/experiments/596555950902332859


2025/05/11 10:06:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5001/#/experiments/596555950902332859/runs/8ed2192cd3894ac5ac05356f8f9e0c19
🧪 View experiment at: http://localhost:5001/#/experiments/596555950902332859


### Register the Model

In [10]:
import mlflow
from mlflow.tracking import MlflowClient

# Set the tracking URI if not already done
mlflow.set_tracking_uri("http://127.0.0.1:5001")  # Ensure this is your active server

# Input Run ID
run_id = input("Please type RunID: ")
model_name = "XGB-Smote"

# The artifact_path must match what you used during log_model
model_uri = f"runs:/{run_id}/model"  # NOT "model_name"

# Register the model
result = mlflow.register_model(model_uri=model_uri, name=model_name)

print(f"Model registered: name={result.name}, version={result.version}")

Please type RunID:  8ed2192cd3894ac5ac05356f8f9e0c19


Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/05/11 10:07:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 3


Model registered: name=XGB-Smote, version=3


Created version '3' of model 'XGB-Smote'.


In [11]:
# model_name = 'XGB-Smote'
# run_id=input('Please type RunID')
# model_uri = f'runs:/{run_id}/model_name'

# with mlflow.start_run(run_id=run_id):
#     mlflow.register_model(model_uri=model_uri, name=model_name)

### Load the Model

In [12]:
model_uri

'runs:/8ed2192cd3894ac5ac05356f8f9e0c19/model'

In [13]:
import mlflow

# Make sure you're pointing to the correct tracking URI
mlflow.set_tracking_uri("http://127.0.0.1:5001")

# Registered model name and version
model_name = "XGB-Smote"
model_version = 2

# Load from the registry
model_uri = f"models:/{model_name}/{model_version}"
loaded_model = mlflow.xgboost.load_model(model_uri)

# Predict on test set
y_pred = loaded_model.predict(X_test)
print(y_pred[:4])

[0 0 0 0]


### Transition the Model to Production

In [14]:
from mlflow.tracking import MlflowClient

production_model_name = "anomaly-detection-prod"
mlflow.set_tracking_uri("http://127.0.0.1:5001")

client = MlflowClient()

# Get latest version (copied earlier)
latest_version = client.get_latest_versions(production_model_name, stages=[])[0].version

# Promote to Production
client.transition_model_version_stage(
    name=production_model_name,
    version=latest_version,
    stage="Production",
    archive_existing_versions=True
)

print(f"Version {latest_version} of model '{production_model_name}' promoted to Production.")

Version 1 of model 'anomaly-detection-prod' promoted to Production.


In [15]:
import mlflow
from mlflow.tracking import MlflowClient

production_model_name = "anomaly-detection-prod"
mlflow.set_tracking_uri("http://127.0.0.1:5001")

# ✅ Use MlflowClient to ensure a version exists in Production stage
client = MlflowClient()
prod_versions = client.get_latest_versions(production_model_name, stages=["Production"])

if not prod_versions:
    raise ValueError(f"No model version in 'Production' stage for model '{production_model_name}'.")

# Get the latest Production version
prod_version = prod_versions[0].version
prod_model_uri = f"models:/{production_model_name}/{prod_version}"

# ✅ Load the model
loaded_model = mlflow.xgboost.load_model(prod_model_uri)

# ✅ Use the model
y_pred = loaded_model.predict(X_test)
print(y_pred[:4])

[0 0 0 0]


Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 